In [1]:
import pandas as pd
from collections import Counter
from functools import reduce

In [ ]:
def process_chunk(chunk):
    cols = chunk.columns
    merge_dicts = lambda ds: reduce(lambda d1, d2: d1 + d2, [Counter(eval(d)) for d in ds])
    chunk = chunk.groupby(cols[0], as_index=False).agg({
        cols[1]: 'mean',
        cols[2]: merge_dicts,
        cols[3]: merge_dicts,
        cols[4]: merge_dicts,
        cols[5]: 'mean'
    })
    for i in range(len(chunk)):
        chunk.at[i, cols[2]] += chunk.at[i, cols[3]] + chunk.at[i, cols[4]]
    chunk.drop([cols[3], cols[4]], axis=1, inplace=True)
    chunk.columns = ['id', 'cat_feature', 'metrics', 'dt_diff']
    chunk.to_csv('data/data_p.csv', mode='a')

In [ ]:
# download & process data
chunk_size = 1000000
i = 1
for chunk in pd.read_csv('data/data.tsv', sep='\t', chunksize=chunk_size):
    process_chunk(chunk)
    print(i / 20, '%')
    i += 1